In [3]:
import os
import shutil
import random
import pathlib
import pandas as pd

In [4]:
datos = pd.read_csv("https://raw.githubusercontent.com/AnIsAsPe/ClassificadorCancerEsofago/master/Datos/ClasesImagenes.csv", usecols=[1,2])

In [5]:
datos['class'] = datos['class_number'] == 1

In [6]:
#Spliting and Copying into Train-Test Folders
target_dir = 'C:\\Users\\conte\\OneDrive\\Escritorio\\Colegio Bourbaki\\ML-AI-WA\\Perceptron\\ImagenesClasificadas\\'
image_path = 'C:\\Users\\conte\\OneDrive\\Escritorio\\Colegio Bourbaki\\ML-AI-WA\\Perceptron\\CarpetaImagenes\\'

count = 0
for image_split in [False, True]:
    labels = list(datos[(datos['class']==image_split)]['image_filename'])
    for label in labels:
        to_dir = pathlib.Path(str(target_dir) + str(image_split) + '/' + str(label))
        if not to_dir.is_dir():
            to_dir.parent.mkdir(parents=True, exist_ok=True)
        from_dir = pathlib.Path(image_path + label)
        shutil.copy2(from_dir, to_dir)
        count += 1

print(f'{count:.0f} images copied ')

5063 images copied 


In [7]:
# Setup data paths
data_path = 'C:\\Users\\conte\\OneDrive\\Escritorio\\Colegio Bourbaki\\ML-AI-WA\\Perceptron\\ImagenesClasificadas\\'

# Create function to split randomly in Train and Test
def train_test_split(image_path=data_path,
               data_splits=['Train', 'Test'], 
               target_classes = [False,True],
               split=0.35,
               seed=42):
    random.seed(42)
    label_splits = {}
    
    # Get labels
    ammount=0
    for data_split in data_splits:
        print(f"[INFO] Creating image split for: {data_split}...")
        image_paths = []
        for target in target_classes:
            labels = list(datos[(datos['class']==target)]['image_filename'])
            if data_split == 'Train':
                ammount = 1 - split
            elif data_split == 'Test':
                ammount = split
            sample = round(ammount*len(labels))
            print(f"[INFO] Getting random set of {sample} images for {data_split}-{target} ...")
            sampled_images = random.sample(labels, k=sample)
            image_paths.append([pathlib.Path(str(image_path + str(target) + '/' + sample_image)) for sample_image in sampled_images])
        image_path_flat = [item for sublist in image_paths for item in sublist]
        label_splits[data_split] = image_path_flat
    
    print('\n','Finish splitting!')

    return label_splits

In [8]:
label_splits = train_test_split(split=0.35)

[INFO] Creating image split for: Train...
[INFO] Getting random set of 955 images for Train-False ...
[INFO] Getting random set of 2336 images for Train-True ...
[INFO] Creating image split for: Test...
[INFO] Getting random set of 514 images for Test-False ...
[INFO] Getting random set of 1258 images for Test-True ...

 Finish splitting!


In [9]:
# Create target directory path
target_dir_name = 'C:\\Users\\conte\\OneDrive\\Escritorio\\Colegio Bourbaki\\ML-AI-WA\\Perceptron\\Imagenes_Clasificadas_Random_Split'
print(f"Creating directory: '{target_dir_name}'")

# Setup the directories & Make the directories
target_dir = pathlib.Path(target_dir_name)
target_dir.mkdir(parents=True, exist_ok=True)


import shutil

count=0
for image_split in label_splits.keys():
    for image_path in label_splits[str(image_split)]:
        dest_dir = target_dir / image_split / image_path.parent.stem / image_path.name
        if not dest_dir.parent.is_dir():
            dest_dir.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy2(image_path, dest_dir)
        count += 1

print(f'{count:.0f} images moved ')

Creating directory: 'C:\Users\conte\OneDrive\Escritorio\Colegio Bourbaki\ML-AI-WA\Perceptron\Imagenes_Clasificadas_Random_Split'
5063 images moved 
